In [1]:
import pandas as pd
#reading the data set
df = pd.read_csv("data/data_topic.csv")

In [2]:
df['dish_reviewed'] = df['dish_reviewed'].fillna('')
df['cuisines'] = df['cuisines'].fillna('')
df['reviews_list'] = df['reviews_list'].fillna('')
df['menu_item'] = df['menu_item'].fillna('')

In [3]:
def get_recommendations_test(name):
    recommended = []
    top10_list = []
    
    name = name.lower()
    #df['name'] = df['name'].str.lower()
    topic_num = df[df['name'].str.lower() == name].Topic.values
    doc_num = df[df['name'].str.lower() == name].Doc.values    
    output_df = df[df['Topic']==topic_num[0]].sort_values('Probability', ascending=False).reset_index(drop=True)

    index = output_df[output_df['Doc']==doc_num[0]].index[0]
    
    top10_list += list(output_df.iloc[index-5:index].index)
    top10_list += list(output_df.iloc[index+1:index+6].index)
    
    #output_df['name'] = output_df['name'].str.name()
    
    for each in top10_list:
        recommended.append(output_df.iloc[each]['name'])
        
    return recommended

In [4]:
get_recommendations_test('Spice Elephant')

['24 Hours Cake Delivery',
 'Beijing Bites',
 'Maashpacheez',
 'Bazaar - Zone By The Park',
 "B'Town Cafe & Restaurant",
 'Beijing Bites',
 'The Hungry Yeti',
 'Kolkata Famous Kathi Rolls',
 'Pramukh Family Restaurant',
 'Health Gourmet']

In [18]:
def get_recommendations(name):
    from sklearn.metrics.pairwise import linear_kernel
    from sklearn.feature_extraction.text import TfidfVectorizer
    import pandas as pd

    recommended = []
    top30_list = []
    top30 = []
    name = name.lower()
    topic_num = df[df['name'].str.lower() == name].Topic.values
    doc_num = df[df['name'].str.lower() == name].Doc.values    
    output_df = df[df['Topic']==topic_num[0]].sort_values('Probability', ascending=False).reset_index(drop=True)
    
    
    cols = ['name','location', 'rest_type', 'rest_type', 'rest_type', 'cuisines', 'dish_reviewed', 'menu_item', 'online', 'table_booking','listed_as', 'listed_in_city']
    
    output_df['combined'] = output_df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    
    # Creating tf-idf matrix
    tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
    tfidf_matrix = tfidf.fit_transform(output_df['combined'])
    
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    scores_df = pd.DataFrame(cosine_similarities)

    index = output_df[output_df['Doc']==doc_num[0]].index[0]
    
    top30_list = list(scores_df.iloc[index].sort_values(ascending = False).iloc[1:31].index)
    
    for each in top30_list:
        if output_df.iloc[each]['name'].lower() != name:
            top30.append(output_df.iloc[each]['name'])
        
    a = [x.lower() for x in top30]

    filtered = output_df[output_df['name'].str.lower().isin(a)]
    filtered_sorted = filtered.sort_values("weighted_rating",ascending=False)
    
    for i in filtered_sorted['name']:
        if recommended.count(i) <= 0:
            recommended.append(i)
        if len(recommended) == 10:
            break

    return recommended

In [19]:
get_recommendations('Spice Elephant')

['Eshanya',
 'Shanghai Court',
 "Chung's Chinese Corner",
 'Peacock',
 'Delicacy',
 'The Purple Pan',
 'Beijing Bites',
 'East Delicious',
 'Hongkong Garden',
 'Happy Singh']

In [20]:
get_recommendations('Pai Vihar')

['Sri Udupi Park',
 'Bengaluru Coffee House',
 'Gokul Kuteera',
 'Sri Lakshmi Vaibhav',
 'The Krishna Grand Xpress',
 'Shanthi Sagar',
 'Sagar Fast Food',
 'Hotel Chandrika',
 'Udupi Aatithya',
 'The Rasaganga']

In [21]:
get_recommendations('Rosewood International Hotel - Bar & Restaurant')

['Cafe Noir',
 'Time Traveller - Sai Vishram',
 'Wild Thyme Restaurant',
 'Little Chef',
 'BISO - Citrus Hotels',
 'Unicorn Bar & Restaurant',
 'The Light House Restaurant',
 'Zodiac - Fortune Select JP Cosmos Hotel',
 'Citrus Cafe - Lemon Tree Hotel',
 'Tayo Restaurant']

In [22]:
get_recommendations('Fujian Express')

['Szechuan Dragon',
 'Delicacy',
 "Chung's Chinese Corner",
 'New Peking',
 'Chung Wah',
 'Beijing Bites',
 'Mountain Spice',
 'Red Lantern',
 "Kenny's Restaurant",
 'Mr Manchurian']

In [23]:
get_recommendations('San Churro Cafe')

['Cafe Max',
 'YORK St.',
 'Little Italy',
 'Little Green Cafe',
 'Green Theory',
 'Mr. Beans - Home Cafe',
 'Tea Villa Cafe',
 'Tea Brew',
 'Roundup Cafe',
 'Eurasia Pasta and Barbeque by Little Italy']

In [24]:
get_recommendations('Woodee Pizza')

['Starbucks',
 "Domino's Pizza",
 'Ovenstory Pizza',
 'Pizza Hut',
 'Cafe Coffee Day',
 'The World Cafe',
 'Cuppa',
 'Cafe Hush',
 'The Studio Cafe',
 '5th Main Cafe']

In [25]:
get_recommendations('Behrouz Biryani')

["Mani's Dum Biryani",
 "Paul's Dum Biryani",
 'Chef All Nite',
 'Hotboxit',
 'The Good Bowl',
 'Aramane Donne Biryani',
 'Ruchi Maayaka',
 'Biryani Treats',
 'Fat Chef Biryani Wale',
 'LSD Cafe']

In [26]:
get_recommendations('Barbeque Nation')

['Andhra Biryanis',
 'Petoo',
 'Atithi',
 'Hotel Greens',
 'The Orchid',
 'Zodiac',
 'Afoozoo - Ginger Hotel',
 'UBQ by Barbeque Nation',
 'Madhura',
 'Mega Bite']